In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [7]:
df = pd.read_csv('../Datasets/voice.csv')
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [8]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,1


In [9]:
X = df.drop('label', axis=1)
y = df['label']

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [12]:
# Train KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [13]:
def predict_gender(input_df: pd.DataFrame):
    if not isinstance(input_df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    # Check column names
    expected_cols = X.columns.tolist()
    if input_df.columns.tolist() != expected_cols:
        raise ValueError(f"Expected columns: {expected_cols}")

    # Scale input
    input_scaled = scaler.transform(input_df)

    # Predict
    pred = knn.predict(input_scaled)
    return le.inverse_transform(pred)


In [14]:
!pip install pandas scikit-learn sounddevice scipy librosa numpy

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------- ----------------- 1.6/2.8 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   --- ------------------------------------ 2.4/30.3 MB 11.7 MB/s eta 0:00:03
   ------ --------------------------------- 4.7/30.3 MB 11.7 MB/s eta 0:00:03
   --------- ------------------------------ 7.1/30.3 MB 11.6 MB/s eta 0:00:02
   ------------ --------------------------- 9.4/30.3 MB 11.6 MB/s eta 0:00:02
   --------------- ------------------------ 11.8/30.3 MB 11.6 MB/s eta 0:00:02
   ------------------ --------------------- 14.2/30.3 MB 11.6 MB/s eta 0:00:02
   --------------------- ------------------ 16.5/30.3 MB 11.6 MB/s eta 0:00:02
   ------------------------ --------------- 18.9/30.3 MB 11.6 MB/s eta 0:00:01
   ---------------------------- ----------- 21.2/30.3 MB 11.6 MB/s eta 0:00:01
  

In [15]:
import sounddevice as sd
from scipy.io.wavfile import write

def record_voice(filename="live_audio.wav", duration=3, fs=44100):
    print("🎙️ Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, recording)
    print(f"✅ Audio saved as {filename}")


In [19]:
import librosa
import numpy as np
import pandas as pd

def extract_features(filename):
    y, sr = librosa.load(filename)

    # Time-domain features
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    energy = np.square(y).mean()

    # Frequency-domain features
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    flatness = librosa.feature.spectral_flatness(y=y)[0]
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]

    # Fundamental frequency estimates
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch = pitches[pitches > 0]

    # Safe pitch-related stats
    if len(pitch) > 0:
        meanfun = pitch.mean()
        minfun = pitch.min()
        maxfun = pitch.max()
    else:
        meanfun = minfun = maxfun = 0

    features = {
        'meanfreq': np.mean(centroid),
        'sd': np.std(centroid),
        'median': np.median(centroid),
        'Q25': np.percentile(centroid, 25),
        'Q75': np.percentile(centroid, 75),
        'IQR': np.percentile(centroid, 75) - np.percentile(centroid, 25),
        'skew': np.mean(bandwidth),
        'kurt': np.std(bandwidth),
        'sp.ent': np.sum(flatness),
        'sfm': np.mean(flatness),
        'mode': np.argmax(np.histogram(y, bins=10)[0]),
        'centroid': np.mean(centroid),
        'meanfun': meanfun,
        'minfun': minfun,
        'maxfun': maxfun,
        'meandom': np.mean(bandwidth),
        'mindom': np.min(bandwidth),
        'maxdom': np.max(bandwidth),
        'dfrange': np.max(bandwidth) - np.min(bandwidth),
        'modindx': np.std(pitch) / np.mean(pitch) if len(pitch) > 0 else 0
    }

    return pd.DataFrame([features])


In [21]:
def predict_from_mic():
    record_voice("live_audio.wav", duration=3)
    features_df = extract_features("live_audio.wav")
    result = predict_gender(features_df)
    print("🗣️ Predicted Gender:", result[0])


In [22]:
predict_from_mic()

🎙️ Recording...
✅ Audio saved as live_audio.wav
🗣️ Predicted Gender: female
